In [33]:
import os
import numpy as np
import pandas as pd
from pandas import to_datetime
from matplotlib import pyplot as plt
import seaborn as sns
import itertools
import warnings
import datetime
from datetime import datetime
warnings.filterwarnings('ignore')

%matplotlib inline

from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
!pip install module
from module import stationarity_adf_test, stationarity_kpss_test
import pmdarima as pm
from pmdarima.model_selection import train_test_split

ERROR: Could not find a version that satisfies the requirement module (from versions: none)
ERROR: No matching distribution found for module


ModuleNotFoundError: No module named 'module'

In [25]:
# raw data 로딩 
raw_data = pd.read_csv(r'C:\\Users\\user\\Desktop\\intern.csv')
raw_data.head()

,Unnamed: 0,ORD_NO,H_YMD,O_YMD,PKG_GOODS_NO,PKG_GOODS_NM,GOODS_NO,GOODS_NM,STD_GSGR_NO_LEV1_NM,ORD_QTY,CANCEL_QTY,RET_QTY,REAL_ORD_QTY,RECVR_ROAD_BASE_ADDR,SALE_PRICE,DISCOUNT_AMT,FINAL_PRICE
0,0,202201095519105,2022-01-11 00:00:00,2022-01-09 12:26:10,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,충청북도 증평군 증평읍 송산로 11 (지평더웰아파트),6900,569.00,6331
1,1,202201105522898,2022-01-11 00:00:00,2022-01-10 10:35:49,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,서울특별시 강남구 논현로85길 52 (역삼동),6900,177.00,6723
2,2,202201115527172,2022-01-12 00:00:00,2022-01-11 11:38:32,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,"서울특별시 송파구 올림픽로4길 42 (잠실동, 우성아파트)",6900,221.00,6679
3,3,202201105523467,2022-01-12 00:00:00,2022-01-10 13:21:51,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,강원도 속초시 도리원길 11-14 (노학동),6900,128.00,6772
4,4,202201105522519,2022-01-11 00:00:00,2022-01-10 08:16:21,단품,단품,1901012353,수제계란말이(350g),반찬,1,0,0,1,"경기도 용인시 수지구 동천로 64 (동천동, 동천마을동문굿모닝힐5차아파트)",6900,157.00,6743


In [26]:
# 자재코드와 자재명을 딕셔너리 형태로 만들기
GOODS_DIC = {1712010310 : '두메산나물비빔밥재료', 14375:	'옛날잡채(500g)', 3273:'채소계란찜(340g)', 13957:'셀프두부조림(600g)', 1901012353:'수제계란말이(350g)',
             13529:'메밀소바(2인분)', 1803010879:'고소한도토리묵무침(360g)', 2012014895:'한돈 제육볶음(700g)', 2204016772:'소고기유니짜장소스(1인분, 200g)', 
             1823:'숙주나물(300g)', 4092:'건표고버섯볶음', 14985:'양장피', 1808011751:'꼬막무침 (260g)', 1903012675:'순살코다리강정(180g)', 
             2010014667:'우삼겹숙주볶음(250g)', 1806011271:'열무비빔밥재료믹스(2인분)', 2103015231:'[심방골주부X더반찬] 시골 돼지짜글이(600g)', 
             2010014707:'고구마 품은 라자냐(450g)', 1805011121:'고사리나물볶음(150g)', 1909013359:'가정집 오징어불고기/셀프(380g)'
             }

In [27]:
GOODS_NO = GOODS_DIC.keys()
GOODS_NM = GOODS_DIC.values()

In [28]:
time_data = raw_data[['H_YMD', 'GOODS_NO', 'REAL_ORD_QTY']]
time_data = time_data.loc[time_data['GOODS_NO'] == 14375]
time_data = time_data.sort_values(by = 'H_YMD')
time_data.drop('GOODS_NO', axis=1, inplace=True)

time_data = time_data.groupby(['H_YMD'], as_index=False)
time_data = time_data.agg({
                           'REAL_ORD_QTY':['count']
                        })
time_data.columns = [
                     'H_YMD', 'transactions',
                     ]

time_data

,H_YMD,transactions
0,2020-01-01 00:00:00,84
1,2020-01-02 00:00:00,106
2,2020-01-03 00:00:00,169
3,2020-01-04 00:00:00,143
4,2020-01-06 00:00:00,106
...,...,...
736,2022-05-26 00:00:00,79
737,2022-05-27 00:00:00,95
738,2022-05-28 00:00:00,86
739,2022-05-30 00:00:00,72


In [29]:
X_train = time_data[time_data['H_YMD'] < '2022-01-01']
X_val = time_data[time_data['H_YMD'] >= '2022-01-01']

print('X_train: ', X_train.shape)
print('X_val: ', X_val.shape)

X_train:  (614, 2)
X_val:  (127, 2)


In [30]:
time_data.set_index('H_YMD', inplace=True)

time_data

,transactions
H_YMD,
2020-01-01 00:00:00,84
2020-01-02 00:00:00,106
2020-01-03 00:00:00,169
2020-01-04 00:00:00,143
2020-01-06 00:00:00,106
...,...
2022-05-26 00:00:00,79
2022-05-27 00:00:00,95
2022-05-28 00:00:00,86


In [31]:
## UDF for KPSS test 
from statsmodels.tsa.stattools import kpss 
import pandas as pd 

def kpss_test(timeseries): 
    print("Results of KPSS Test:") 
    kpsstest = kpss(timeseries, regression="c", nlags="auto") 
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"] ) 
    
    for key, value in kpsstest[3].items(): 
        kpss_output["Critical Value (%s)" % key] = value 
    print(kpss_output)

kpss_test(time_data)

Results of KPSS Test:
Test Statistic           0.17
p-value                  0.10
Lags Used               14.00
Critical Value (10%)     0.35
Critical Value (5%)      0.46
Critical Value (2.5%)    0.57
Critical Value (1%)      0.74
dtype: float64


In [32]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(time_data['transactions'], model='additive')

fig = plt.figure()
fig = result.plot()
fig.set_size_inches(20,15)

ValueError: You must specify a period or x must be a pandas object with a PeriodIndex or a DatetimeIndex with a freq not set to None